# HyperSegmentedModel with TDStone2

## 1 - Connect to Vantage

In [1]:
import teradataml as tdml
import json
tdml.__version__

'20.00.00.03'

In [2]:
import tdstone2
tdstone2.__version__

'0.1.5.3'

In [3]:
with open('../connections/me.json','r') as f:
    Param_ = json.load(f)

with open('../connections/csae2.json','r') as f:
    Param = json.load(f)

with open('../connections/csae.json','r') as f:
    Param_ = json.load(f)

with open('../connections/vantage24.json','r') as f:
    Param_ = json.load(f)

tdml.create_context(**Param)

Engine(teradatasql://:***@sto-xo0saiv0ot1sxmj2.env.clearscape.teradata.com?DATABASE=demo_user&user=DEMO_USER)

## 2 - Setup of the sto framework

In [4]:
from tdstone2.utils import cleanup
                   
cleanup(schema=Param['database'])
         

c:\Users\dm250067\AppData\Local\anaconda3\envs\vantage_python\Lib\site-packages\teradataml\dbutils\dbutils.py:333: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pddf = pd.read_sql(query, tdmlctx.td_connection.connection)
c:\Users\dm250067\AppData\Local\anaconda3\envs\vantage_python\Lib\site-packages\teradataml\dbutils\dbutils.py:339: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vtdf = pd.read_sql(vtquery, tdmlctx.td_connection.connection)
Processing tables/views: 100%|██████████| 26/26 [00:05<00:00,  4.59table/view/s, Dropped TABLE demo_user.TDS_MAPPER_2cf3e19f_a506_482e_b5e0_46f5d91f069b]                      


In [5]:
from tdstone2.tdstone import TDStone
sto = TDStone(schema_name = Param['database'], SEARCHUIFDBPATH = Param['user'])
sto.setup()

2025-01-08 11:25:55 - INFO - Starting setup: Creating repositories.
2025-01-08 11:25:55 - INFO - Code repository created.
2025-01-08 11:25:55 - INFO - Model repository created.
2025-01-08 11:25:56 - INFO - Mapper repository created.
2025-01-08 11:25:56 - INFO - Hypermodel repository created.
2025-01-08 11:25:56 - INFO - Feature engineering process repository created.
2025-01-08 11:25:56 - INFO - Environment detected as Vantage Enterprise. Starting file installation with PushFile.
Installing tds_vector_embedding_lake: 100%|██████████| 9/9 [00:00<00:00, 1368.40it/s]
2025-01-08 11:26:01 - INFO - Setup complete.


In [6]:
sto.PushFile()

Installing tds_vector_embedding_lake: 100%|██████████| 9/9 [00:00<00:00, 2417.47it/s]


tdml.execute_sql('SET SESSION SEARCHUIFDBPATH = "demo_user"')
query = f"""
    SELECT TOP 100
        EXECUTION_TIME AS TD_TIMECODE,
        'TDS_STO_SCORING_27739a86_2d82_47f0_8126_e48bcb94012b' AS ID_PROCESS,
        Partition_ID,
        ID,
        FEATURE_NAME,
        FEATURE_VALUE,
        FEATURE_TYPE,
        STATUS,
        ID_TRAINED_MODEL
        FROM SCRIPT(
        ON demo_user.TDS_ON_CLAUSE_SCORING_90770514_d7b3_4aa3_abee_0af3a92c5b21
        --ON DEMO_USER.TDS_VOLATILE_ON_CLAUSE_SCORING_27739a86_2d82_47f0_8126_e48bcb94012b
        PARTITION BY Partition_ID --,  STO_MODEL_ID ,FOLD
        
        ORDER BY ID
        SCRIPT_COMMAND(
        'tdpython3 ./demo_user/tds_scoring.py  | awk -F"	" "NF == 10"'
        )
        /*
        SCRIPT_COMMAND(
        'tdpython3 ./demo_user/tds_scoring.py'
        )
        */
        RETURNS(
        'Partition_ID INTEGER', 
        'ID VARCHAR(2000)'
        ,	'FEATURE_NAME     VARCHAR(2000)'
        ,	'FEATURE_VALUE    VARCHAR(2000)'
        ,	'FEATURE_TYPE     VARCHAR(20000)'
        ,	'STATUS           VARCHAR(20000)'
        ,   'ID_MODEL         VARCHAR(36)'
        ,	'MODEL_TYPE       VARCHAR(255)'
        ,	'ID_TRAINED_MODEL VARCHAR(36)'
        ,   'EXECUTION_TIME   TIMESTAMP(6) WITH TIME ZONE'
        )
        CHARSET('LATIN')
        ) AS d;
"""
tdml.execute_sql(query).fetchall()

In [7]:
tdml.remove_context()

True